## what is <strong>ATTENTION</strong>
in a model to perform well, it should be able to focus on the most relevant parts of the input.which is called <strong>attention</strong>.
for example in a sentence like "A cat sat on the mat and jumped over the dog", the model should be able to focus on the words "cat", "sat", "on", "the", "mat" and "jumped" and etc. By doing so it makes it understand that the actual sentence is about a cat jumping over dog not a dog jumping over cat or anything else.
- it is essential for language translation models to learn how to focus on the most relevant parts of the input.
- for exmple we are converting timi sanchai xau,bhai? to english it cant be you fine are,brother? but you are fine,bhai?
- here needs attention mechanism.
- now in transformers, the model uses attention to focus on the most relevant parts of the input.but before transformers,there used to be RNNs which are neural network + encoder decoder which encoder generated context vector and decoder used that context vector to generate the output.
- Attention Mechanism are 
  1.simplified attention
  2.multihead attention
  3.self attention

## now lets understand RNNs and How they work.
1. RNNs are a type of neural network that is used to process sequential data.
2. when a single word comes in encoder in encoder a hidden state is generated.
3. and it continuously updated until the end of the sentence.
4. then in final hidden state there is context of the whole sentence.which is kind of memory of the sentence.
5. this final hidden state is used by the decoder to generate the output.
6. the output is generated as one word at a time using the context which now helps generating better output.
- But there was problem with this that the decoder would use only the final hidden state to generate the output. which could be a problem if the sentence is long.
- to solve this problem transformers introduced attention mechanism.

# Before Transformers there was Bahdanau attention which would modify the RNNs in which the decoder would use the encoder previous hidden state to generate the output.
- for example there is a sentence like "A cat sat on the mat and jumped over the dog".
  while decoder is translating word jumped,it may give attention to the word cat also whch makes model understand that the actual sentence is about a cat jumping over dog not a dog jumping over cat or anything else.
- this is called Bahdanau attention.
* upto now we have understood how decoding happens using the encoders previous hidden state to generate the next word.
* Lets understand What is self-Attention.
  - Self attention is technique in which the mechanism's ability to weigh attention weight within the single input sequence. 
  

# simplified Attention mechanism

- for example consider we have a sintence that "Your Journey Starts with a Question".
- after vector and input embedding we get for each word/token a vector embedding a long dimensional vector.
- now these vectors while plotting would give us a graph in visual representation.
- now to get the context vector for a token with any other token we need to calculate the similarity between the two vectors.
- this similarity is given by the dot product of the two vectors.
  ![attention](./public/atten.png)

  

In [ ]:
# pip install torch

In [ ]:
import torch 

ModuleNotFoundError: No module named 'torch'

In [ ]:
def attention_calculation(query,input):
    # print(input.shape)
    attention_scores = torch.empty(len(input))
    for i in range(len(input)):
        # print(len(input))
        attention_scores[i] = torch.dot(query, input[i])
    return attention_scores


In [ ]:
# coding in python
inputs = torch.tensor(
    [
        [0.1, 0.2, 0.3, 0.4],
        [0.2, 0.3, 0.4, 0.5],
        [0.3, 0.4, 0.5, 0.6],
        [0.4, 0.5, 0.6, 0.7],
        [0.5, 0.6, 0.7, 0.8],
        [0.6, 0.7, 0.8, 0.9],
    ]
)
# print(inputs[1])

attention_score = attention_calculation(inputs[1],inputs)#calculates attention score with query and all the input tokens but we need to now normalize it to get better results later for use

print(attention_score)

tensor([0.4000, 0.5400, 0.6800, 0.8200, 0.9600, 1.1000])


## normalization makes it between 0 and 1.
- one way is to divide each score by the sum of all scores 
- but we havve problem because we dont want to consider a token attention if it is not near to the query token.
- we use softmax to normalize the scores
- in this way we get the attention score between 0 and 1 for each token by exponential of atention score divided by sum of all attention scores which make model understand that no need to give attention to a token if it is not near to the query token 


In [ ]:
def soft_max_naive(x):
    return torch.exp(x)/torch.exp(x).sum(dim=0)

normalized_attention_score = soft_max_naive(attention_score)
print(normalized_attention_score)
print(normalized_attention_score.sum(dim=0))

tensor([0.1142, 0.1313, 0.1510, 0.1737, 0.1999, 0.2299])
tensor(1.)


In [ ]:
#it is better to use pytorch softmax function

normalized_attention_score_2 = torch.softmax(attention_score, dim=0)
print(normalized_attention_score_2)
print(normalized_attention_score_2.sum(dim=0))

tensor([0.1142, 0.1313, 0.1510, 0.1737, 0.1999, 0.2299])
tensor(1.)


# now we want to calculate the context vector
- it is calculated by multiplying the normalized attention score with the input tokens's input embedding vector
- then after multiplying we get a vector of same dimension as the input embedding vector for each tokens of the input sequence which is context vector for the particular query with tokens of input sequence. 
- now these context vectors are summed to get the final context vector for the query token.
- now the final context vector is used to generate the output by using the decoder.
  ![context](./public/cntxt.png)
![attention](./public/ctxt.png)

In [ ]:
#code 
def calculate_context_vector(inputs, normalized_attention_score_2):
    context_vector = torch.empty(inputs.shape[1])
    for i,x_i in enumerate(inputs):
        context_vector+= normalized_attention_score_2[i] * x_i
    return context_vector


context_vector = calculate_context_vector(inputs, normalized_attention_score_2)
print(context_vector)#this is the context vector for the query token 'Journey'

tensor([0.3903, 2.3653, 0.5903, 0.6903])


In [ ]:
attention = inputs @ inputs.T
print(attention)
normalized_attention = torch.softmax(attention, dim=-1)
print(normalized_attention)

tensor([[0.3000, 0.4000, 0.5000, 0.6000, 0.7000, 0.8000],
        [0.4000, 0.5400, 0.6800, 0.8200, 0.9600, 1.1000],
        [0.5000, 0.6800, 0.8600, 1.0400, 1.2200, 1.4000],
        [0.6000, 0.8200, 1.0400, 1.2600, 1.4800, 1.7000],
        [0.7000, 0.9600, 1.2200, 1.4800, 1.7400, 2.0000],
        [0.8000, 1.1000, 1.4000, 1.7000, 2.0000, 2.3000]])
tensor([[0.1279, 0.1414, 0.1562, 0.1727, 0.1908, 0.2109],
        [0.1142, 0.1313, 0.1510, 0.1737, 0.1999, 0.2299],
        [0.1014, 0.1214, 0.1454, 0.1740, 0.2083, 0.2494],
        [0.0897, 0.1118, 0.1393, 0.1735, 0.2163, 0.2695],
        [0.0790, 0.1025, 0.1329, 0.1723, 0.2235, 0.2899],
        [0.0693, 0.0935, 0.1262, 0.1704, 0.2300, 0.3105]])


In [ ]:
row_2 = normalized_attention[1]
print(row_2)
print(row_2.sum(dim=0))

tensor([0.1142, 0.1313, 0.1510, 0.1737, 0.1999, 0.2299])
tensor(1.)


In [ ]:
#calculate context vector for each query token
context_vector = normalized_attention @ inputs
print(context_vector)

tensor([[0.3790, 0.4790, 0.5790, 0.6790],
        [0.3903, 0.4903, 0.5903, 0.6903],
        [0.4015, 0.5015, 0.6015, 0.7015],
        [0.4123, 0.5123, 0.6123, 0.7123],
        [0.4228, 0.5228, 0.6228, 0.7228],
        [0.4330, 0.5330, 0.6330, 0.7330]])


## How to convert Input Embedding to Context Vector ??
- it is done by 1st we calculate the attention score for each token of the input sequence.
- this is done by calculating the dot product of the query token with each token of the input sequence.
- then we normalize the attention score using softmax.
- then we calculate the context vector by 1st multiplying the normalized attention score of each token (with respect to query token) with the input embedding vector of each token.
- then we sum all these context vectors to get the final context vector for the query token.
- this is fed to the decoder to generate the output.

# now how do we even get the attention score for each token of the input sequence and the context vector?
- to get attention score with each other in a sequence we do matrix multiplication of input embedding matrix to transpose of input embedding matrix.which returns the attention score for each token of the input sequence with respect to each other token of the input sequence.
- now to get the normalized attention score we use softmax.
- now to get the context vector for each token of the input sequence we do matrix multiplication of normalized attention score with input embedding matrix.
* boom we get the context vector for each token of the input sequence.that 1st row of this vectr gives the context vector for the 1st token and so on... also remember that 1st row of the input embedding matrix gives the input embedding vector for the 1st token.so we just converted all the input embedding vectors to context vectors.

** REMEMBER THAT WE NEED TO USE TRAINABLE WEIGHTS FOR THIS. **

## No we look into the self attention
- before we looked into the simplified attention in which we did not use any trainable weights.
- trainable weights are used in self attention to provide better results.
- here we will have 3 trainable matrices - query matrix, key matrix and value matrix.
- we multiply each of these matrix to the input embedding matrix to get the query matrix, key matrix and value matrix.
- then we calculate the attention score for each token of the input sequence with respect to each other token of the input sequence.
- then we normalize the attention score using softmax.
- then

![self](./public/self.png)

![self2](./public/self_attention.png)

In [ ]:
# coding self attention
inputs = torch.tensor(
    [
        [0.1, 0.2, 0.3, 0.4],#your
        [0.2, 0.3, 0.4, 0.5],#journey
        [0.3, 0.4, 0.5, 0.6],#starts
        [0.4, 0.5, 0.6, 0.7],#with
        [0.5, 0.6, 0.7, 0.8],#a
        [0.6, 0.7, 0.8, 0.9],#question
    ]
)

In [ ]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2

w_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
w_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
w_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

In [ ]:
v_key = x_2 @ w_key
v_query = x_2 @ w_query
v_value = x_2 @ w_value
print(v_key)
print(v_query)
print(v_value)

tensor([0.8529, 0.6798])
tensor([0.9397, 0.5642])
tensor([0.8143, 0.7649])


In [ ]:
keys = inputs @ w_key
queries = inputs @ w_query
values = inputs @ w_value
print(keys.shape)
print(queries.shape)
print(values.shape)
# print(keys)
# print(queries)
# print(values)

torch.Size([6, 2])
torch.Size([6, 2])
torch.Size([6, 2])


In [ ]:
attention_score_2 = queries @ keys.T

In [ ]:
#now we will use these 3 matrices instead of the input embeddings to calculate the attention score

#to get the attention score for 2nd token "Journey" we will dot product the query vectors 2nd row with all the key vectors


attention_score = queries @ keys.T
print(attention_score)
d_k = keys.shape[-1]
normalized_attention_score = torch.softmax(attention_score/d_k**0.5, dim=-1)#across the columns of the matrix
print(normalized_attention_score)
context_vector = normalized_attention_score @ values
print(context_vector)

tensor([[0.6168, 0.8560, 1.0952, 1.3344, 1.5736, 1.8128],
        [0.8542, 1.1850, 1.5158, 1.8466, 2.1775, 2.5083],
        [1.0916, 1.5140, 1.9365, 2.3589, 2.7813, 3.2038],
        [1.3290, 1.8430, 2.3571, 2.8712, 3.3852, 3.8993],
        [1.5664, 2.1720, 2.7777, 3.3834, 3.9891, 4.5948],
        [1.8037, 2.5010, 3.1984, 3.8957, 4.5930, 5.2903]])
tensor([[0.1048, 0.1241, 0.1469, 0.1740, 0.2061, 0.2441],
        [0.0859, 0.1085, 0.1371, 0.1732, 0.2188, 0.2765],
        [0.0696, 0.0938, 0.1265, 0.1705, 0.2298, 0.3098],
        [0.0558, 0.0803, 0.1155, 0.1661, 0.2389, 0.3436],
        [0.0443, 0.0680, 0.1044, 0.1602, 0.2458, 0.3773],
        [0.0349, 0.0571, 0.0935, 0.1531, 0.2507, 0.4106]])
tensor([[1.2600, 1.2205],
        [1.2994, 1.2608],
        [1.3368, 1.2990],
        [1.3718, 1.3348],
        [1.4042, 1.3679],
        [1.4339, 1.3983]])


In [ ]:
#lets now impliment context vector from input embeddings using nn.modules

class selfAttention_v1(torch.nn.Module):
    def __init__(self, input_dim, context_dim):
        super().__init__()
        self.w_query = torch.nn.Parameter(torch.rand(input_dim, context_dim))
        self.w_key = torch.nn.Parameter(torch.rand(input_dim, context_dim))
        self.w_value = torch.nn.Parameter(torch.rand(input_dim, context_dim))
    
    def forward (self,input):
        keys = input @ self.w_key
        queries = input @ self.w_query
        values = input @ self.w_value
        attention_score = queries @ keys.T
        d_k = keys.shape[-1]
        normalized_attention_score = torch.softmax(attention_score/d_k**0.5, dim=-1)
        context_vector = normalized_attention_score @ values
        return context_vector


model = selfAttention_v1(inputs.shape[-1], 2)#input columns and parameter's rows should be same as we will be performing matrix multiplication
context_vector = model(inputs)
print(context_vector)

tensor([[0.7303, 1.3201],
        [0.7526, 1.3610],
        [0.7739, 1.3998],
        [0.7939, 1.4363],
        [0.8125, 1.4703],
        [0.8297, 1.5016]], grad_fn=<MmBackward0>)


In [ ]:
#lets now impliment context vector from input embeddings using nn.modules

class selfAttention_v2(torch.nn.Module):
    def __init__(self, input_dim, context_dim):
        super().__init__()
        self.w_query = torch.nn.Linear(input_dim, context_dim, bias=False)
        self.w_key = torch.nn.Linear(input_dim, context_dim, bias=False)
        self.w_value = torch.nn.Linear(input_dim, context_dim, bias=False)
    
    def forward(self, inputs):
        # call the Linear modules instead of trying to matrix-multiply them
        keys = self.w_key(inputs)
        queries = self.w_query(inputs)
        values = self.w_value(inputs)
        attention_score = queries @ keys.T
        d_k = keys.shape[-1]
        normalized_attention_score = torch.softmax(attention_score / d_k**0.5, dim=-1)
        context_vector = normalized_attention_score @ values
        return context_vector


torch.manual_seed(0)
model = selfAttention_v2(inputs.shape[-1], 2)#input columns and parameter's rows should be same as we will be performing matrix multiplication
context_vector = model(inputs)
print(context_vector)

tensor([[-0.1236,  0.4347],
        [-0.1236,  0.4343],
        [-0.1236,  0.4339],
        [-0.1235,  0.4334],
        [-0.1235,  0.4330],
        [-0.1235,  0.4325]], grad_fn=<MmBackward0>)


## Now the causal mechanism
- it is mechanism in which the attention of the token before the token considered is only taken 
- it makes the attention matrix a lower triangular matrix and then normalized the matrix
# steps to follow:
 1. 1st we will get the input embedding matrix for the tokens as earlier
 2. then we get the query, key and value weight with torch.nn.Parameterwith input_dim = input embedding matrix's columns and wanted context vector's columns
 3. we matrix multiply each of these matrices to the input embedding matrix to get the query matrix, key matrix and value matrix.
 4. then we get the attention score for each token of the input sequence with respect to each other token of the input sequence by matrix multiplication of query matrix to transpose of key matrix.
 5. after this we will mask the attention score to avoid data leakages by setting the upper triangular entries to -inf and then mask the attention score wwith this upper matrix.
 6. now to get the normalized attention score we use softmax to attention score/d_k**0.5 where d_k is the number of columns of the key matrix.after that we may use nn.Dropout to avoid overfitting.
 7. now to get the context vector for each token of the input sequence we do matrix multiplication of normalized attention score with value matrix.

![causal](./public/causal.png)

# Dropout
![causal](./public/dropout.png)




In [ ]:
context_len = attention_score.shape[0]
masked = torch.tril(torch.ones(context_len,context_len))
attention_masked = masked * attention_score


In [ ]:
total = attention_masked.sum(dim=1,keepdim=True)
normalized_attention_masked = attention_masked/total
print(normalized_attention_masked)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4189, 0.5811, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2403, 0.3333, 0.4263, 0.0000, 0.0000, 0.0000],
        [0.1582, 0.2194, 0.2806, 0.3418, 0.0000, 0.0000],
        [0.1128, 0.1564, 0.2000, 0.2436, 0.2872, 0.0000],
        [0.0848, 0.1175, 0.1503, 0.1830, 0.2158, 0.2486]])


In [ ]:
# ensure no present token is influenced by future tokens;this avoids the data leakages
# before softmax set the upper-triangular entries to -inf
mask = torch.triu(torch.ones(context_len, context_len), diagonal=1).bool()
masked = attention_score.masked_fill(mask, float("-inf"))
print(masked)

tensor([[0.6168,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.8542, 1.1850,   -inf,   -inf,   -inf,   -inf],
        [1.0916, 1.5140, 1.9365,   -inf,   -inf,   -inf],
        [1.3290, 1.8430, 2.3571, 2.8712,   -inf,   -inf],
        [1.5664, 2.1720, 2.7777, 3.3834, 3.9891,   -inf],
        [1.8037, 2.5010, 3.1984, 3.8957, 4.5930, 5.2903]])


In [ ]:
normalized_masked_attention = torch.softmax(masked/keys.shape[-1]**0.5,dim=-1)
print(normalized_masked_attention)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4418, 0.5582, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2401, 0.3236, 0.4363, 0.0000, 0.0000, 0.0000],
        [0.1336, 0.1922, 0.2765, 0.3977, 0.0000, 0.0000],
        [0.0712, 0.1092, 0.1676, 0.2572, 0.3948, 0.0000],
        [0.0349, 0.0571, 0.0935, 0.1531, 0.2507, 0.4106]])


In [ ]:
# now we apply dropout so to avooid overfitting 
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5)
dropout_attn = dropout(normalized_attention_masked)
print(dropout_attn)

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4807, 0.6667, 0.8527, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4388, 0.5612, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3128, 0.0000, 0.4872, 0.0000, 0.0000],
        [0.0000, 0.2350, 0.3006, 0.3661, 0.4316, 0.0000]])


In [ ]:
#now causal attention class
class Causal_attention(torch.nn.Module):
     def __init__(self, input_dim, context_dim,qkv_bias=False, attention_len=6):
        super().__init__()
        self.w_query = torch.nn.Linear(input_dim, context_dim, bias=qkv_bias)
        self.w_key = torch.nn.Linear(input_dim, context_dim, bias=qkv_bias)
        self.w_value = torch.nn.Linear(input_dim, context_dim, bias=qkv_bias)
        self.register_buffer("mask",torch.triu(torch.ones(attention_len,attention_len),diagonal=1))
        
     def forward(self, inputs):
      b, num_token_seq, token_dim = inputs.shape
      keys = self.w_key(inputs)
      queries = self.w_query(inputs)
      values = self.w_value(inputs)

      attention_score = queries @ keys.transpose(1,2)
      masked = attention_score.masked_fill(self.mask[:num_token_seq, :num_token_seq].bool(), float('-inf'))
      
      normalized_attention_score = torch.softmax(masked / (keys.shape[-1]**0.5), dim=-1)
      dropout = torch.nn.Dropout(0.0)
      attention = dropout(normalized_attention_score)
      
      context_vector = attention @ values
      return context_vector

        



NameError: name 'torch' is not defined

: 

In [ ]:
batch = torch.stack((inputs,inputs),dim=0
)
print(batch)

# pass attention_len as a keyword (positional args cannot come after keyword args)
model = Causal_attention(batch.shape[-1], 3, qkv_bias=True, attention_len=batch.shape[1])
context_vector = model(batch)
print(context_vector)


tensor([[[-0.2252,  0.2267, -0.3400],
         [-0.2489,  0.2630, -0.3248],
         [-0.2724,  0.2989, -0.3097],
         [-0.2957,  0.3344, -0.2948],
         [-0.3188,  0.3697, -0.2800],
         [-0.3417,  0.4047, -0.2653]],

        [[-0.2252,  0.2267, -0.3400],
         [-0.2489,  0.2630, -0.3248],
         [-0.2724,  0.2989, -0.3097],
         [-0.2957,  0.3344, -0.2948],
         [-0.3188,  0.3697, -0.2800],
         [-0.3417,  0.4047, -0.2653]]], grad_fn=<UnsafeViewBackward0>)


In practical terms, implementing multi-head attention involves creating multiple instances of the self-attention mechanism, each with its own weights, and then combining their outputs

In code, we can achieve this by implementing a simple MultiHeadAttentionWrapper class that stacks multiple instances of our previously implemented CausalAttention module:

In multihead attention we just create multiple key,query and value matrixes and then perform matrix multiplication with each of these matrixes to get the query matrix, key matrix and value matrix.

then find context vector for each pairs these 
at last we just add these context vectors along columns to get the final output context vector.

![multihead](./public/multi.png)


In [ ]:
class MultiHeadAttentionWrapper(torch.nn.Module):

    def __init__(self, d_in, d_out, context_length, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = torch.nn.ModuleList(
            [Causal_attention(d_in, d_out, qkv_bias, attention_len=context_length) 
             for _ in range(num_heads)]
        )

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)


In [ ]:
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape) 

torch.manual_seed(123)
context_length = batch.shape[1] # This is the number of tokens = 6
d_in, d_out = inputs.shape[-1], 3
mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, num_heads=2)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

torch.Size([2, 6, 3])
tensor([[[ 0.3326,  0.5659, -0.3132,  0.0752,  0.4566,  0.2729],
         [ 0.3456,  0.5650, -0.2237,  0.0313,  0.5977,  0.3053],
         [ 0.3440,  0.5604, -0.2000,  0.0178,  0.6413,  0.3138],
         [ 0.3103,  0.4941, -0.1606,  0.0089,  0.5729,  0.2785],
         [ 0.2430,  0.4287, -0.1643,  0.0071,  0.5566,  0.2514],
         [ 0.2648,  0.4316, -0.1375,  0.0023,  0.5363,  0.2508]],

        [[ 0.3326,  0.5659, -0.3132,  0.0752,  0.4566,  0.2729],
         [ 0.3456,  0.5650, -0.2237,  0.0313,  0.5977,  0.3053],
         [ 0.3440,  0.5604, -0.2000,  0.0178,  0.6413,  0.3138],
         [ 0.3103,  0.4941, -0.1606,  0.0089,  0.5729,  0.2785],
         [ 0.2430,  0.4287, -0.1643,  0.0071,  0.5566,  0.2514],
         [ 0.2648,  0.4316, -0.1375,  0.0023,  0.5363,  0.2508]]],
       grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 6])


For example, if we use this MultiHeadAttentionWrapper class with two attention heads (via num_heads=2) and CausalAttention output dimension d_out=2, this results in a 4- dimensional context vectors (d_out*num_heads=4)

### IMPLEMENTING MULTI-HEAD ATTENTION WITH WEIGHT SPLITS

<div class="alert alert-block alert-warning">
    
Instead of maintaining two separate classes, MultiHeadAttentionWrapper and
CausalAttention, we can combine both of these concepts into a single
MultiHeadAttention class. 

Also, in addition to just merging the
MultiHeadAttentionWrapper with the CausalAttention code, we will make some other
modifications to implement multi-head attention more efficiently.
</div>

<div class="alert alert-block alert-warning">
    
In the MultiHeadAttentionWrapper, multiple heads are implemented by creating a list
of CausalAttention objects (self.heads), each representing a separate attention head.


The CausalAttention class independently performs the attention mechanism, and the
results from each head are concatenated.

In contrast, the following MultiHeadAttention
class integrates the multi-head functionality within a single class. 


It splits the input into
multiple heads by reshaping the projected query, key, and value tensors and then combines
the results from these heads after computing attention.
</div>

<div class="alert alert-block alert-info">

Step 1: Reduce the projection dim to match desired output dim

Step 2: Use a Linear layer to combine head outputs

Step 3: Tensor shape: (b, num_tokens, d_out)

Step 4: We implicitly split the matrix by adding a `num_heads` dimension. Then we unroll last dim: (b,
num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)

Step 5: Transpose from shape (b, num_tokens, num_heads, head_dim) to (b, num_heads, num_tokens, head_dim)

Step 6: Compute dot product for each head

Step 7: Mask truncated to the number of tokens

Step 8: Use the mask to fill attention scores

Step 9: Tensor shape: (b, num_tokens, n_heads, head_dim)

Step 10: Combine heads, where self.d_out = self.num_heads * self.head_dim

Step 11: Add an optional linear projection
</div>

In [ ]:
class MultiHeadAttention(torch.nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        # assert (d_out % num_heads == 0), 
        #     "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = torch.nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = torch.nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

In [ ]:
torch.manual_seed(123)

# Define the tensor with 3 rows and 6 columns
inputs = torch.tensor(
    [[0.43, 0.15, 0.89, 0.55, 0.87, 0.66],  # Row 1 = token1
     [0.57, 0.85, 0.64, 0.22, 0.58, 0.33],  # Row 2
     [0.77, 0.25, 0.10, 0.05, 0.80, 0.55]]  # Row 3
)

batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape) 

batch_size, context_length, d_in = batch.shape
d_out = 6
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

torch.Size([2, 3, 6])
tensor([[[ 0.1569, -0.0873,  0.0210,  0.0215, -0.3243, -0.2518],
         [ 0.1117, -0.0547,  0.0406, -0.0213, -0.3251, -0.2993],
         [ 0.1196, -0.0491,  0.0318, -0.0635, -0.2788, -0.2578]],

        [[ 0.1569, -0.0873,  0.0210,  0.0215, -0.3243, -0.2518],
         [ 0.1117, -0.0547,  0.0406, -0.0213, -0.3251, -0.2993],
         [ 0.1196, -0.0491,  0.0318, -0.0635, -0.2788, -0.2578]]],
       grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 3, 6])


## Lets dive deepper into the LLM architecture
we will implement the GPT model